In [1]:
#Grab Data
import yfinance as yf

#Usual Suspects
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# plt.style.use("seaborn-v0_8-deep")

import plotly.express as px
from pypfopt.plotting import plot_weights
import seaborn as sns

# Use PyPortfolioOpt for Calculations
from pypfopt import EfficientFrontier, objective_functions
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel, plotting
from pypfopt import DiscreteAllocation

In [2]:
#Create a Portfolio
df = pd.read_csv('hackathon_sample_v2.csv')
symbols = df.stock_ticker.unique()
symbols = symbols[~pd.isnull(symbols)]
symbols = symbols.tolist()

In [3]:
porfolioFile = "yfinance_download.csv"

if not os.path.isfile(porfolioFile):
    portfolio = yf.download(symbols, start="2000-01-01", end="2023-12-31")['Adj Close']
    portfolio.to_csv(porfolioFile, index=True)
else:
    portfolio = pd.read_csv(porfolioFile)

portfolio.head()


[******                13%                       ]  463 of 3565 completedFailed to get ticker 'NMG/A' reason: Expecting value: line 1 column 1 (char 0)
[**********************55%*                      ]  1969 of 3565 completedFailed to get ticker 'JW/A' reason: Expecting value: line 1 column 1 (char 0)
[**********************60%****                   ]  2156 of 3565 completedFailed to get ticker 'BRK/A' reason: Expecting value: line 1 column 1 (char 0)
[**********************69%********               ]  2470 of 3565 completedFailed to get ticker 'FCE/A' reason: Expecting value: line 1 column 1 (char 0)
[**********************83%***************        ]  2947 of 3565 completedFailed to get ticker 'AGR/A' reason: Expecting value: line 1 column 1 (char 0)
[**********************87%*****************      ]  3106 of 3565 completedFailed to get ticker 'BF/B' reason: Expecting value: line 1 column 1 (char 0)
[**********************89%******************     ]  3190 of 3565 completedFailed to g

Ticker,A,AA,AAI,AAL,AAN,AAON,AAP,AAPL,AAS,AAWW,...,ZMH,ZNGA,ZNT,ZNTL,ZOMX,ZQK,ZRAN,ZS,ZTS,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03 00:00:00+00:00,44.290768,72.063606,NaN,NaN,NaN,0.609822,NaN,0.859423,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04 00:00:00+00:00,40.907444,72.397484,NaN,NaN,NaN,0.635232,NaN,0.786965,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05 00:00:00+00:00,38.369949,76.571045,NaN,NaN,NaN,0.640878,NaN,0.798481,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06 00:00:00+00:00,36.908974,75.569412,NaN,NaN,NaN,0.643701,NaN,0.729382,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07 00:00:00+00:00,39.984726,75.346786,NaN,NaN,NaN,0.621115,NaN,0.763932,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
portfolio.to_csv(porfolioFile, index=False)

In [5]:
#SP500 ETF Benchmark
market_prices = yf.download("SPY", start='2000-01-01', end='2023-12-31')["Adj Close"]
market_prices.head()

[*********************100%***********************]  1 of 1 completed


Date
2000-01-03    97.506668
2000-01-04    93.693573
2000-01-05    93.861176
2000-01-06    92.352676
2000-01-07    97.716209
Name: Adj Close, dtype: float64

In [7]:
marketCapsFile = 'market_caps.csv'

if not os.path.isfile(marketCapsFile):
    mcaps = {}

    for t in symbols:
        try:
            stock = yf.Ticker(t)
            mcaps[t] = stock.info["marketCap"]
        except:
            pass
    market_caps = pd.Series(mcaps)
    market_caps.to_csv(marketCapsFile, index=False)
else:
    mcaps = pd.read_csv(marketCapsFile)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SUNW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SUNW&crumb=PKnnE6GSlfe
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ADLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ADLA&crumb=PKnnE6GSlfe
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CMCS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CMCS&crumb=PKnnE6GSlfe
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FPL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo

{'ORCL': 459426856960,
 'MSFT': 3190037938176,
 'TW': 26615502848,
 'TROW': 24367802368,
 'HON': 133357961216,
 'ADPT': 614958272,
 'T': 154805485568,
 'ADM': 29037563904,
 'ARC': 146183984,
 'CF': 15315174400,
 'BC': 5484613632,
 'UIS': 395905728,
 'MKL': 20352239616,
 'DELL': 82351153152,
 'MEC': 415689120,
 'KO': 307423051776,
 'CG': 15445293056,
 'UBSI': 4989581824,
 'XRAY': 5288808448,
 'FAST': 41121710080,
 'DTE': 25937534976,
 'ALEX': 1422661120,
 'DD': 36367990784,
 'ETN': 130333966336,
 'XOM': 520033239040,
 'BMCS': 3696407,
 'DY': 5705567744,
 'GD': 83716612096,
 'GE': 205650247680,
 'GM': 54026833920,
 'IR': 39545466880,
 'IBM': 203546083328,
 'AM': 7267041280,
 'OLN': 5405000192,
 'PCG': 51833262080,
 'PEP': 233397026816,
 'MO': 87341400064,
 'AMGN': 177812930560,
 'S': 7692682240,
 'AAPL': 3456956104704,
 'SUN': 7312612352,
 'AMAT': 162226208768,
 'TX': 7180946944,
 'TKR': 5886405632,
 'TR': 2321233152,
 'UCL': 48364164,
 'MRO': 15427782656,
 'VMC': 32997830656,
 'Z': 1558

In [ ]:
marketCapsFile = 'market_caps.csv'
market_caps = pd.Series(mcaps)
market_caps.to_csv(marketCapsFile, index=False)

# Getting Priors

In [ ]:
#Calculate Sigma and Delta to get implied market returns
#Ledoit-Wolf is a particular form of shrinkage, where the shrinkage coefficient is computed using O?
S = risk_models.CovarianceShrinkage(portfolio).ledoit_wolf()

delta = black_litterman.market_implied_risk_aversion(market_prices)
delta

In [ ]:
#Visualize the Covariant Correlation
sns.heatmap(S.corr(), cmap='coolwarm')

In [ ]:
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)
market_prior

In [ ]:
#What am I looking at here?
market_prior.plot.barh(figsize=(10,5));

# Integrating Views

In [32]:
#You don't have to provide views on all the assets
viewdict = {
    'AAPL':0.10,
    'MSFT':0.10,
    'META':0.05,
    'AMZN':0.30,
    'XOM':0.02,
    'UNH':0.01,
    'JNJ':0.15,
    'V':0.09,
    'HD':0.16,
    'ABBV':0.07,
    'KO':0.01,
    'DIS':-0.23,
    'T':0.16,
    'UPS':0.10,
    'LMT':-0.09,
    'CAT':0.30,
    'F':0.16,
    'MAR':-0.08,
    'O':0.30,
    'HSY':-0.26
}

bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict)

## Creating Confidences

In [33]:
intervals = [
    (0, 0.25),
    (0.1, 0.4),
    (-0.1, 0.15),
    (-0.05, 0.1),
    (0.15, 0.25),
    (-0.1, 0),
    (0.1, 0.2),
    (0.08, 0.12),
    (0.1, 0.9),
    (0, 0.3),
    (0, 0.25),
    (0.1, 0.4),
    (-0.1, 0.15),
    (-0.05, 0.1),
    (0.15, 0.25),
    (-0.1, 0),
    (0.1, 0.2),
    (0.08, 0.12),
    (0.1, 0.9),
    (0, 0.3),
]

In [ ]:
variances = []
for lb, ub in intervals:
    sigma = (ub - lb)/2
    variances.append(sigma ** 2)

print(variances)
omega = np.diag(variances)

# Calculate Posterior Estimate Returns

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
im = ax.imshow(omega)

# We want to show all ticks...
ax.set_xticks(np.arange(len(bl.tickers)))
ax.set_yticks(np.arange(len(bl.tickers)))

ax.set_xticklabels(bl.tickers)
ax.set_yticklabels(bl.tickers)
plt.show()

In [36]:
# We are using the shortcut to automatically compute market-implied prior
bl = BlackLittermanModel(S, pi="market", market_caps=mcaps, risk_aversion=delta,
                        absolute_views=viewdict, omega=omega)

In [ ]:
# Posterior estimate of returns
ret_bl = bl.bl_returns()
ret_bl 

In [ ]:
rets_df = pd.DataFrame([market_prior, ret_bl, pd.Series(viewdict)],
             index=["Prior", "Posterior", "Views"]).T
rets_df

In [ ]:
rets_df.plot.bar(figsize=(12,8));

In [ ]:
S_bl = bl.bl_cov()
plotting.plot_covariance(S_bl);

# Portfolio Allocation

In [ ]:
ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()
weights

In [ ]:
pd.Series(weights).plot.pie(figsize=(9,9));

In [ ]:
# Maximum Sharpe
ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()

plot_weights(weights)
ef.portfolio_performance(verbose = True, risk_free_rate = 0.009)